In [ ]:
# Required for importing modules from parent directory
import os
import sys

import soccer_twos

from src.config import N_GAMES
from src.utils import shape_rewards

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
CHECKPOINT_DIR = "src\runs\out\checkpoints\checkpoints.zip"

In [ ]:
import random

import numpy as np


def evaluate_agent(agent, n_games=N_GAMES):
    env = soccer_twos.make(render=True, time_scale=1, quality_level=5, worker_id=3)

    for i in range(n_games):
        obs = env.reset()
        done = False
        scores = {}
        i = 0
        while not done:
            i += 1
            actions = {}
            for j in range(4):
                actions[j], _ = agent.predict(np.array(obs[j]))

            next_obs, reward, done, info = env.step(actions)
            print(
                "[Agent 0] Reward: ",
                reward[0],
                " Score: ",
                shape_rewards(info, 0),
                "Action: ",
                actions[0],
            )
            done = done["__all__"]
            shape_rewards(info, 0)
            obs = next_obs

    env.close()

In [ ]:
from pathlib import Path

from stable_baselines3 import PPO

best_agent = PPO.load(Path("src/runs/out/checkpoints/bestmodel_ppo.zip"))
# Evaluate the trained agent
evaluate_agent(best_agent, n_games=5)

In [ ]:
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import load_results

df = load_results("out/logs/plain/")
df.head()